# Simple approach to getting data

In this notebook we walk through a slightly simpler method of querying OOI 
data, where you don't need to know quite so many details to access the data.
 This methodology is from [the ooi repo](https://github.com/oceanobservatories/ooi-data-explorations/tree/master/python)
 and it includes a summary yaml file that simplifies and groups more than 
 100 nodes in the network in a way that makes queries more straightforward 
 for most users. 
 
Rather than defining site, node, sensor, method, and stream, users only need
 to specify site, assembly, instrument, and method.

In [1]:
from yooink import DataFetcher
from yooink import ooi_seconds_to_datetime
import os

In [2]:
# Initialize the API client with your username and token.
# 
# This assumes you have your OOI API information stored as environment 
# variables "OOI_USER" and "OOI_TOKEN". Adjust this as needed! You can 
# hard-code these but do this with caution (ie don't push to a public repo!)
username = os.getenv('OOI_USER')
token = os.getenv('OOI_TOKEN')

In [3]:
# Example setup for requesting met data from an instrument on the Oregon
# shelf surface mooring. 

site = 'CE02SHSM'           # OOI site designator
assembly = 'buoy'       # Assembly grouping name
instrument = 'metbk'        # OOI instrument class 
method = 'telemetered'      # data delivery method

# You need to specify at least a start date and/or
# stop date, or use the deployment number
start = '2019-04-01'       # data for April 2019 ...
stop = '2019-05-30'        # ... through May 2019
# deploy = 9                 # The Spring 2019 deployment number

In [4]:
fetcher = DataFetcher(username=username, token=token)
ds = fetcher.get_dataset(
    site=site,
    assembly=assembly,
    instrument=instrument,
    method=method,
    start=start,
    stop=stop,
)

Using cached URL for request: CE02SHSM_SBD11_06-METBKA000_telemetered_metbk_a_dcl_instrument_2019-04-01T04:00:00.000Z_2019-05-30T04:00:00.000Z


Processing files: 100%|██████████| 2/2 [00:12<00:00,  6.05s/it]


In [5]:
# The result is an xarray dataset
ds 

<xarray.Dataset> Size: 29MB
Dimensions:                                   (time: 86447, string14: 14,
                                               string5: 5)
Coordinates:
  * time                                      (time) float64 692kB 3.763e+09 ...
Dimensions without coordinates: string14, string5
Data variables: (12/90)
    northward_wind_velocity_qc_executed       (time) uint8 86kB 29 29 ... 29 29
    air_temperature_qc_results                (time) uint8 86kB 29 29 ... 29 29
    northward_wind_velocity                   (time) float32 346kB -0.2 ... -...
    precipitation                             (time) float32 346kB 18.88 ... ...
    sea_surface_conductivity_qc_executed      (time) uint8 86kB 29 29 ... 29 29
    sea_surface_temperature_qartod_executed   (time) <U2 692kB '11' ... '11'
    ...                                        ...
    sea_surface_temperature_qartod_results    (time) uint8 86kB 1 1 1 ... 1 1 1
    precipitation_qc_executed                 (time) uint8 86kB 29 29 ... 29 29
    northward_wind_velocity_qc_results        (time) uint8 86kB 29 29 ... 29 29
    met_relwind_speed                         (time) float64 692kB 0.6616 ......
    met_netsirr_qc_results                    (time) uint8 86kB 29 29 ... 29 29
    relative_humidity_qartod_executed         (time) <U2 692kB '11' ... '11'
Attributes: (12/70)
    node:                               SBD11
    comment:                            
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             metbk_a_dcl_instrument
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                44.63558
    lon:                                -124.3039

In [6]:
# Extracting specific parts from the xarray dataset; convert the time to 
# python datetime (the time coordinate is in units of seconds since 
# 1900-01-01 so we're converting it to something more usable for analysis.)
wind_dir = ds['met_relwind_direction']
wind_speed = ds['met_relwind_speed']
time = ds['time']
if time.attrs['units'] == 'seconds since 1900-01-01 0:0:0':
    time_py = ooi_seconds_to_datetime(ds['time'].data)


In [8]:
[x for x in list(ds.data_vars) if "wind" in x.lower()]

['northward_wind_velocity_qc_executed',
 'northward_wind_velocity',
 'northward_wind_velocity_qartod_executed',
 'eastward_wind_velocity_qartod_results',
 'met_relwind_direction',
 'met_relwind_direction_qc_results',
 'met_relwind_direction_qc_executed',
 'eastward_wind_velocity_qc_results',
 'met_windavg_mag_corr_east',
 'eastward_wind_velocity',
 'northward_wind_velocity_qartod_results',
 'met_windavg_mag_corr_north',
 'eastward_wind_velocity_qc_executed',
 'met_relwind_speed_qc_results',
 'eastward_wind_velocity_qartod_executed',
 'met_relwind_speed_qc_executed',
 'northward_wind_velocity_qc_results',
 'met_relwind_speed']